<a href="https://colab.research.google.com/github/egynk3/simpleNeuralNetwork/blob/main/notebooks/SimpleNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing relevant libs
import numpy as np               # Numerical operations
import pandas as pd              # Data handling (e.g., read CSV files)
from sklearn.model_selection import train_test_split   # Split data into train/test
from sklearn.preprocessing import StandardScaler       # Normalize features
#pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
#evaluation and vizualization
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#importing data
df = pd.read_csv('data.csv')
#normalize data
#split into sets training validation and test sets

########################################################
#scale inputs and outputs
#normalize
########################################################


########################################################
##define architecture
#input layer
#hidden layers
#output layer
#drop out + batch normalization for training stability
########################################################


########################################################
#define loss funciton
#define optimizer
#optional metric tracker r^2
########################################################

########################################################
#feed training data into model
#use va;odatopm data to monitor overfitting
#adjust number of epochs and batch size as needed
########################################################


########################################################
#test model on unseen ddata
#compute mse and r^2 to quantify performance
#vizualize predicted vs actual h

#########################################################
#save trained model weights
#function that takes new values and returns predicted


SyntaxError: incomplete input (ipython-input-39773322.py, line 12)